In [ ]:
sample_size = 1000               #this is used to load the training_data
true_unknown_distribution = "wiggly1"             
is_agnostic = False
specific_hypothesis = 2

epsilon = 0.05
hypothesis_class_cardinality = 10000


In [ ]:
data_path = '/content/drive/My Drive/Simulations in Stat learning/data/'
distributions_path = '/content/drive/My Drive/Simulations in Stat learning/distributions/'
videos_path = '/content/drive/My Drive/Simulations in Stat learning/videos/'

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Helper functions

In [ ]:
#Helper Functions

def get_labels(x,y,distribution):
    if distribution == "circle":
        z = np.where(x**2 + y**2 + 2*0.9*x + 2*0.5*y - 0.2 > 0 , 1 , 0)  
        
    elif distribution == "linear":
        h_labeling = np.array([0.7,0.5])
        
        temp = np.r_[x,y]  #---for analytical and empirical comparison
        temp = temp.T
        z = np.where(np.matmul(temp,h_labeling) - 0> 0.0, 1, 0)
        
    elif distribution =="wiggly1":
        z = np.where(3*y**3 +x**2 -0.2*y**2 +  0*x - 0.01   >0, 1, 0)
        
    elif distribution =="wiggly2":
        z = np.where(30*x**10 + 2*y**10 -700*x**7 + 20*y**5 -1*x**2 + 0.02   >0, 0, 1)
        
    elif distribution == "l1":
        z1 = np.where(abs(x)+abs(y)>0.15,1,0)
        z2 = np.where(abs(x)+abs(y)>0.4,1,0)

        z = np.logical_xor(z1,z2)
    
    else :
        filename = distributions_path + distribution + ".png"
        im = plt.imread(filename)
        image = np.round(np.sum(im,2)/3)

        x_ = np.floor((x + 0.4)*1000)
        y_ = np.floor((y + 0.4)*1000)

        labels = []
        for i,j in zip(x_.astype(int),y_.astype(int)):

            label = image[i,j]
            labels.append(label)

        z = np.abs(1 - np.array(labels))
    
    
        
    return z

In [ ]:
def errors_on_hypothesis_set(data,hypothesis):
    intermediate = np.matmul(data[:,:2], hypothesis)
    predictions = np.where(intermediate > 0.0, 1, 0)
    labels = data[:,2]

    E_out = np.logical_xor(predictions.T,labels)
    true_errors = 1 - np.sum(E_out,axis=1)/E_out.shape[1]
    
    return true_errors,E_out[:8,:],predictions[:,:8]

In [ ]:
def growth_function(n,vc_dim,vc_infinite):
  #this can be made tighter
  if vc_infinite:
    return np.power(2,n)
  else:
    return 4*(np.power(n,vc_dim)+1)

def exponential_part(n,e):
  x = (-0.125)*e*e*n
  return np.exp(x)
  #return np.power(ep,x)

#Guarentees of the sample of size n being e-representative if there are finite number of hypothesis or there are infinite number of hypothesis with 
#a finite vc dimension

def get_guarantee_infinite(n,e,vc_dim,vc_infinite):
  gf = growth_function(2*n,vc_dim,vc_infinite)
  ex = exponential_part(n,e)
  #what would be the prodict of these guys if vc dim is infinite
  return gf*ex,gf,ex 

def get_guarantee_finite(n, e , h):
    return 2.0 *h * np.exp(-2 * n * e * e)

def get_plot_infinite(e,vc_dim,vc_infinite):
  guarantees = []
  growth_functions = []
  exponential_parts = []
  sample_sizes = []

  if vc_infinite:
    start = 1
    end = 40
    diff = 1
  else:
    start = 30
    end =  10000
    diff = 1

  for i in range(start,end,diff):
    sample_sizes.append(i)

    delta,gf,ex = get_guarantee_infinite(i,e,vc_dim,vc_infinite)
    growth_functions.append(gf)
    exponential_parts.append(ex)
    guarantees.append(delta)

  fig1,axs = plt.subplots(1,3)
  axs[0].plot(sample_sizes,growth_functions)
  axs[1].plot(sample_sizes,exponential_parts)
  axs[2].plot(sample_sizes,guarantees)

def get_plot_finite(e,h):
  guarantees = []
  sample_sizes = []
  for i in range(30,10000,1):
    sample_sizes.append(i)

    delta = get_guarantee_finite(i,e,h)
    guarantees.append(delta)

  fig2,axs = plt.subplots(1,1)
  axs.plot(sample_sizes,guarantees)

# Knobs

In [ ]:
n = 300 #100,400,4000,10000,20000.
e = 0.1
vc_dim = 2
bound = get_guarantee_infinite(n,e,vc_dim,False)
print(bound)

(989699.310616115, 1440004, 0.6872892787909721)


In [ ]:
#Make a pattern out of these
n = sample_size
distribution = true_unknown_distribution
agnostic = is_agnostic  

if agnostic == False:
  name = distribution
else:
  name = 'agnostic'


In [ ]:
data = np.load(data_path + "data_"+name+'_'+str(40000)+".npy")


In [ ]:
#Constants relevant to the notebook
e = epsilon
vc_dim = 2

In [ ]:
num_hypothesis = hypothesis_class_cardinality
hypothesis = np.random.uniform(-1,1,[2,num_hypothesis])

h8 = np.array([[0.5,0.5],
               [0.5,0],
               [0.5,-0.5],
               [-0.5,0.5],
               [-0.5,0],
               [-0.5,-0.5],
               [0,0.5],
               [0,-0.5]])
hypothesis[:,:8] = h8.T



In [ ]:
true_errors,E_out,p_true = errors_on_hypothesis_set(data,hypothesis)

#Starting Analyisis

In [ ]:
#Works well for n=1000,e=0.05,h = 10000 .


union_bound = get_guarantee_finite(n,e,num_hypothesis)
vc_bound = get_guarantee_infinite(n,e,vc_dim,vc_infinite=False) #Todo remove this redundancy

In [ ]:
union_bound

134.75893998170935

In [ ]:
vc_bound

(11705852.989608785, 16000004, 0.7316156289466418)

In [ ]:



e_representetive = 0
trials = 1000
for i in range(trials):
  #Make this efficient
  interpolated_points = np.random.uniform(-0.4,0.4,[1,n,2])
  interpolation_intermediate = np.matmul(interpolated_points,hypothesis)
  interpolation_predictions = np.where(interpolation_intermediate > 0.0, 1, 0) # I can bias term here
 

  interpolation_labels = get_labels(interpolated_points[:,:,0],interpolated_points[:,:,1],distribution).reshape(1,n,1)
  #print(interpolation_predictions.shape)
  #print(interpolation_labels.shape)
  interpolation_in_error = 1 - np.sum(np.logical_xor(interpolation_predictions, interpolation_labels),axis=1)/interpolated_points.shape[1]
  interpolation_diff = np.abs(interpolation_in_error - true_errors)


  x = np.where(interpolation_diff>0.05)
  if x[0].size == 0:
    e_representetive+=1

print(1 - (e_representetive/trials))

# For circle decision boundry. Linear gives similar results? **OVERLAPS**

#According to empirical tests,probability of 1000 point sample not being e-representative:(**This depends on the distribution!!??**)
#**0.012**,0.008 , 0.019 , 0.01

#According to hoeffdings bound,probabilitiy of 1000 point sample not being e-representative is less than or equal to(**This is distribution agnostic**):
#134
#****
# For skull decision boundry

#According to empirical tests,probability of 1000 point sample not being e-representative:(**This depends on the distribution!!??**)  **0.049**,0.033,0.039,0.0385,0.032

#According to hoeffdings bound,probabilitiy of 1000 point sample not being e-representative is less than or equal to(**This is distribution agnostic**):
#**134**


0.027000000000000024
